In [41]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import csv
from fileinput import filename
import os
import numpy as np
import glob
import configparser
from random import random, shuffle
import operator
import h5py
import pandas as pd
import warnings
from datetime import datetime
from imblearn.under_sampling import RandomUnderSampler
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from utils import make_or_restore_model
import generator as generator
# from ProgressBar import ProgressBar

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

path_2017 = os.path.abspath('D:\TFG\datasets\IDS2018\preprocessed_IDS2017/')
path_2018 = os.path.abspath('D:\TFG\datasets\IDS2018\preprocessed_IDS2018/')
dataset = "/TRAIN"
path_2017 = Path(f"{path_2017}{dataset}/SEPARATED/DDoS")
path_2018 = Path(f"{path_2018}{dataset}_2017/SEPARATED/DDoS")
# path_2017.mkdir(parents=True, exist_ok=True)
# path_2018.mkdir(parents=True, exist_ok=True)

datetimeformat = "%d/%m/%Y %I:%M:%S %p"

In [43]:
import importlib
importlib.reload(generator)

params = configparser.ConfigParser()
params._interpolation = configparser.ExtendedInterpolation()
params.read('./config.ini')

directory = 'DIRECTORIES_' + params['RUN_CONFIG']['dataset']
path_logs = os.path.abspath(params[directory]['logs'])

(model, startingEpoch) = make_or_restore_model(params=params)
val_dataset = generator.input_fn(data_path=os.path.abspath(params[directory]["TRAIN"]), validation=True)

Restoring from D:\TFG\datasets\IDS2018\TensorflowModel\DDoS_2017\NO_FEATURES_3_Flag\ckpt\weights.92-0.665.tf


In [44]:
result = model.evaluate(val_dataset,
          batch_size=1,
          use_multiprocessing=True)

Opening file in generator: D:\TFG\datasets\IDS2018\preprocessed_IDS2017\TRAIN\SEPARATED\DDoS\dataset.csv
   1020/Unknown - 42s 31ms/step - loss: 0.7004 - categorical_accuracy: 0.5001 - specificity_at_sensitivity: 0.9643 - rec_0: 0.0000e+00 - pre_0: 0.0000e+00 - rec_1: 0.0000e+00 - pre_1: 0.0000e+00 - rec_2: 0.0000e+00 - pre_2: 0.0000e+00 - rec_3: 0.0000e+00 - pre_3: 0.0000e+00 - rec_4: 0.0000e+00 - pre_4: 0.0000e+00 - rec_5: 0.0000e+00 - pre_5: 0.0000e+00 - rec_6: 0.0000e+00 - pre_6: 0.0000e+00 - rec_7: 0.0000e+00 - pre_7: 0.0000e+00 - rec_8: 0.0000e+00 - pre_8: 0.0000e+00 - rec_9: 0.0000e+00 - pre_9: 0.0000e+00 - rec_10: 0.0000e+00 - pre_10: 0.0000e+00 - rec_11: 0.0000e+00 - pre_11: 0.0000e+00 - rec_12: 0.0000e+00 - pre_12: 0.0000e+00 - rec_13: 0.0000e+00 - pre_13: 0.0000e+00 - rec_14: 1.0000 - pre_14: 0.5001 - macro_F1: 0.0444 - weighted_F1: 0.3334
Nº graphs:  1022
Total counter:  204400
Last counter:  0
1022/1022 [==============================] - 42s 31ms/step - loss: 0.7004 - cate

In [28]:

result = model.predict(val_dataset)

Opening file in generator: D:\TFG\datasets\IDS2018\preprocessed_IDS2017\EVAL\SEPARATED\DDoS\dataset.csv
    246/Unknown - 6s 22ms/step
Nº graphs:  249
Total counter:  49800
Last counter:  0
250/250 [==============================] - 6s 22ms/step


In [29]:
result.shape

(49800, 15)

In [34]:
df_2017 = pd.read_csv(f'{path_2017}/dataset.csv', header=None, names=generator.features)

In [36]:
df_2017["Label_N"] =  [generator.attacks_dict[l] for l in df_2017["Label"]]

In [30]:
predicted_classes = [list(pred).index(max(pred)) for pred in result]

In [31]:
pd.Series( predicted_classes).value_counts()

14    49800
dtype: int64

In [39]:
import sklearn.metrics as sk
sk.f1_score(df_2017["Label_N"].to_numpy(), predicted_classes, average="weighted")

0.33487396119808044